# 入社課題： 桜の開花日予測

***

## 0. 桜の開花メカニズム

一年を通じての桜の花の生態は、おおまかに生成・休眠・生長・開花の４つの時期に分けることができる。 <br>
　　　　生成： つぼみの生成生長（夏〜秋） <br>
　　　　休眠： つぼみの生長停止時期（晩秋〜冬） <br>
　　　　生長： つぼみの生長（晩冬〜春） <br>
　　　　開花： 花が咲いてから散るまで（春） <br>
十分休眠した桜のつぼみは晩冬から春先にかけてのある日を境に休眠を終え（これを <b>休眠打破</b> という）、その後しばらく暖かい時期を過ごして生長すると春に開花する。たとえば一年中気温が20℃を超えるような地域では冬季の休眠が十分に取れず、桜は花を咲かせることができない。 <br> <br>
本課題では、上記のメカニズムを念頭に置いて東京における桜の開花日を予測するモデルを作成する。以下、予測モデルについては <br>
　テスト年： 1966年・1971年・1985年・1994年・2008年 <br>
　トレーニング年： 1961~2017年のうちの上記以外の全ての年 <br>
として、トレーニング年のデータで予測モデルの作成、テスト年のデータでモデルの性能を評価せよ。

### 問題 0-1
気象庁HPから東京における1961~2017年の桜の開花日のデータを取得せよ。

***
## 1. 「600$^\circ$Cの法則」の検証

桜の開花日を予測する最も簡単な方法として、600$^\circ$Cの法則というものが知られている。これは予測したい年の2月1日から毎日の最高気温を記録して足しあげていき、その積算値が初めて600$^\circ$Cを超えた日を開花日とするものである。ただし、この600$^\circ$Cという数値は日本全国のあらゆる地点で簡単に開花日を予測できるように設定された値で、厳密には地域によって異なる値をとる。ここでは東京における「600$^\circ$Cの法則」の妥当性を検証する。

### 問題 1-1
トレーニング年について2月1日から実際の開花日まで日毎の最高気温を足しあげ、その積算値を年に対してプロットせよ。
また、得られた積算値を平均した値   $T_{\rm{mean}}$ を求め、本当に600$^\circ$Cを開花日予測の閾値とすべきか確認せよ。

### 問題 1-2
問題1-1で求めた $T_{\rm{mean}}$ を用いてテスト年の桜開花日を予測し、実際の開花日との誤差を評価せよ。
また、 $T_{\rm{mean}}$ を閾値として用いた予測は 600$^\circ$Cを閾値とした予測に比べ、どれほど精度が向上するかについて決定係数 $R^2$ を用いて評価せよ。

***
## 2. 温度変換日数法による開花日の回帰予測

桜の開花日は、気温に大きく影響を受ける。金野・杉原ら(1986)は化学反応速度論の立場に基き、気温が生物の活性にもたらす影響を評価する指数として、ある温度下における1日分の活性量が他の温度下では何日分の活性量に相当するのかを表す温度変換日数($DTS$: the number of Days Transformed to Standard temperature)という概念を導入した。温度変換日数は以下の２つの仮定から成る。<br>
 
１つ目の仮定は、生物の成長速度(化学反応速度) $k$ が以下のアレニウスの式 <br> <br>

$$
k = A \exp{ \Bigl(- \frac{E_{a}}{RT}} \Bigr)
$$ <br>

に従うというものである。ここで $A > 0$ は反応頻度因子(定数)、$E_{a} \rm [J/mol]$ は活性化エネルギー、$R = 8.314 \rm [J/K・mol]$ は気体定数、$T\rm[K]$ は絶対温度である。 <br>
　２つ目の仮定は、ある特定の反応量に達するのに要した期間 $t$ と、そのときの反応速度 $k$ の積は異なる温度条件の下でも一定となるというもので、 <br> <br>
 
$$
tk = t'k' = t''k'' = \cdots = \rm{const}
$$ <br>

と表せる。上の２つの式を用いると、温度変換日数を求める以下の式を導くことができる。<br> <br>
$$
t_{s} = \exp \Bigl( \frac{E_{a}(T_{i, j} - T_{s})}{RT_{i, j}T_{s}}\Bigr)
$$ <br>

本課題では$T_{i, j}$としてその日の平均気温を用い、標準温度は$T_{s} = 17^\circ\rm{C}$とする。ある年 $j$ の休眠打破日(起算日)を $D_{j}$、開花日を $BD_{j}$ とすると、その年の休眠打破から開花までに要した温度変換日数 $DTS_{j}$ は次のように計算できる。 <br> <br>
$$
DTS_{j} = \sum_{i=D_{j}}^{BD_{j}} t_{s} = \sum_{i=D_{j}}^{BD_{j}} \exp \Bigl( \frac{E_{a}(T_{i, j} - T_{s})}{RT_{i, j}T_{s}}\Bigr)
$$ <br>

これにより、例えば $x$ 年間の $DTS_{j}$ の平均値 $DTS_{\rm{mean}}$ は <br> <br>
$$
\begin{align}
DTS_{\rm{mean}} &= \frac{1}{x} \sum_{j}^{x} DTS_{j} \\
&= \frac{1}{x} \sum_{j}^{x} \sum_{i=D_{j}}^{BD_{j}} \exp \Bigl( \frac{E_{a}(T_{i, j} - T_{s})}{RT_{i, j}T_{s}}\Bigr)
\end{align}
$$ <br> <br>
により算出される。$DTS_{\rm{mean}}$ と $E_{a}$ は年によって変化しない定数であり、この問題では次の手順でこれら２つの値をトレーニング年のデータから算出することを目指す。（[小元・青野(1989)による速度論的手法](https://www.jstage.jst.go.jp/article/agrmet1943/45/1/45_1_25/_article/-char/ja/)）。 <br>
　　1.　各年 $j$ の起算日 $D_{j}$ を求める。 <br>
　　2.　各年 $j$ において$E_{a}$ を微小に変化させながら、それに対応する $DTS_{j}$ を求める。さらに各$E_{a}$に対する全てのトレーニング年の $DTS_{j}$ の平均 $DTS_{\rm{mean}}$を算出する。<br>
　　3.　活性化エネルギー $E_{a}$ を一定の値に固定して、あるトレーニング年 $j$ について $t_{s}$ を起算日 $D_{j}$ から積算していき、積算値が初めて  $DTS_{\rm{mean}}$ を超える日をその年の開花予測日 $BD_{j}^{\rm{pred}}$ とする。全てのトレーニング年について活性化エネルギー $E_{a}$ を微小に変化させながら、開花予測日 $BD_{j}^{\rm{pred}}$ と実際の開花日 $BD_{j}$ の誤差(平均二乗誤差)を最小にするような $E_{a}$ と、そのときの $DTS_{\rm{mean}}$ の組を求める。 <br>
　　4.　1.〜3.で求めた起算日 $D_{j}$、活性化エネルギー $E_{a}$ および温度変換日数の平均値 $DTS_{\rm{mean}}$ から、テスト年の桜開花日を予測する。

### 問題 2-1
[塚原・林(2012)](http://www.airies.or.jp/journal_17-1jpn.html)によると、ある年の休眠打破日（起算日）$D_{j}$ は次の重回帰式で与えられる。 <br> <br>
$$
D_{j} = 136.75 - 7.689 \phi + 0.133 \phi^{2} -1.307\ln L + 0.144T_{F} + 0.285T_{F}^2
$$ <br>
ここで変数は $\phi:$ 緯度[°N]、 $L:$ 海岸からの距離[km]、 $T_{F}:$ その年の1~3月の平均気温の平均値で、$\ln x$ は自然対数である。この重回帰式から東京の緯度 $\phi = $ 35度40分、海岸からの距離 $L=4 \rm{km}$を用いて、各年（トレーニング・テスト年全て）の $D_{j}$ を求め、プロットせよ。 <br>
（※ 問題1の方法ではこの起算日を毎年一律で2月1日としている）

### 問題 2-2
活性化エネルギー $E_{a}$ を $5 \sim 40\, \rm{kcal}$ の範囲で $1\,\rm{kcal}$ おきに様々に変えて ($E_{a} = 5, 6, 7, \cdots, 40 \,\rm{kcal}$) トレーニング年の各年の温度変換日数 $DTS_{j}$ を計算し、横軸に $E_{a}$ をとって各年の $DTS_{j}$ 、および全てのトレーニング年の温度変換日数の平均値 $DTS_{\rm{mean}}$ をプロットせよ。 $t_{s}$の計算の際には各パラメータの単位に注意せよ。

### 問題 2-3
問題2-2で求めた各 $E_{a}$ に対して、 $DTS_{\rm{mean}}$ からトレーニング年の各年における桜の開花日を予測し、実際の開花日との平均二乗誤差を $E_{a}$ に対しプロットせよ。また、その平均二乗誤差を最小にするような $E_{a}^{*}$ を求めよ。

### 問題 2-4
問題2-1で求めた起算日 $D_{j}$ と、問題2-2で求めた平均温度変換日数 $DTS_{\rm{mean}}$、問題2-3で求めた最適活性化エネルギー $E_{a}^{*}$ を用いてテスト年の開花日を予測し、実際の開花日との誤差を決定係数 $R^2$ で評価せよ。

### 問題 2-5
開花日予測の精度をさらに上げるにはどうすればよいか、改善案がある場合はそれを記し、どの程度精度が向上するかを述べよ。

## 3. ニューラルネットを用いた開花日予測

### 問題 3-1
ニューラルネットワークモデルを自由に組み立て、そのモデルをトレーニング年で学習させてテスト年の開花日を予測し、実際の開花日との誤差を決定係数 $R^2$ で評価せよ。
ただし、データは配布した tokyo.csv および問題0-1で入手した桜の開花日データのみを用いよ。ニューラルネットワークのフレームワークは自由に選んでよい(chainer, TensorFlow, ...)。

### 問題 3-2
問題1の600$^\circ$Cの法則、問題2の温度変換日数法、上で作ったニューラルネットワークによる各予測開花日を実際の開花日に対して散布図プロットし、それぞれのモデルの決定係数 $R^2$ を求めモデル同士の性能を評価せよ。
3つのモデルの違いや精度の差について議論せよ。

## 4. 桜の開花現象について

### 問題 4-1
東京の桜において、過去60年間のデータに基づいて、休眠打破日 $D_{j}$ と開花日 $BD_{j}$ の変化について述べよ。